## lab 5

In [1]:
%%file foo.c
#include <stdio.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int main(int argc, char *argv[]){
	int ret;
	ret = mkdir("mynewdir", S_IRUSR | S_IRGRP | S_IROTH | S_IXUSR | S_IXGRP | S_IXOTH);

//	ret = rmdir("mynewdir");
	
	//ret = symlink("mynewdir", "hellodir");
	
	ret = unlink("hellodir");
	return 0;
}

Writing foo.c


In [3]:
!g++ foo.c -o foo

In [5]:
!./foo

## lab 6

In [17]:
%%file notify.c
#include <stdio.h>
#include <stdlib.h>
#include <errno.h>
#include <sys/types.h>
#include <linux/inotify.h>

#define EVENT_SIZE  ( sizeof (struct inotify_event) )
#define EVENT_BUF_LEN     ( 1024 * ( EVENT_SIZE + 16 ) )

int main(){
  int length, i = 0;
  int fd;
  int wd;
  char buffer[EVENT_BUF_LEN];

  /*creating the INOTIFY instance*/
  fd = inotify_init();

  /*checking for error*/
  if ( fd < 0 ) {
    perror( "inotify_init" );
  }

  /*adding the “/tmp” directory into watch list. Here, the suggestion is to validate the existence of the directory before adding into monitoring list.*/
  wd = inotify_add_watch( fd, "/forNotify", IN_CREATE | IN_DELETE );

  /*read to determine the event change happens on “/tmp” directory. Actually this read blocks until the change event occurs*/ 

  length = read( fd, buffer, EVENT_BUF_LEN ); 

  /*checking for error*/
  if ( length < 0 ) {
    perror( "read" );
  }  

  while ( i < length ) {     struct inotify_event *event = ( struct inotify_event * ) &buffer[ i ];     if ( event->len ) {
      if ( event->mask & IN_CREATE ) {
        if ( event->mask & IN_ISDIR ) {
          printf( "New directory %s created.\n", event->name );
        }
        else {
          printf( "New file %s created.\n", event->name );
        }
      }
      else if ( event->mask & IN_DELETE ) {
        if ( event->mask & IN_ISDIR ) {
          printf( "Directory %s deleted.\n", event->name );
        }
        else {
          printf( "File %s deleted.\n", event->name );
        }
      }
    }
    i += EVENT_SIZE + event->len;
  }
  /*removing the “/tmp” directory from the watch list.*/
   inotify_rm_watch( fd, wd );

  /*closing the INOTIFY instance*/
   close( fd );
}

Overwriting notify.c


In [14]:
!mkdir forNotify

mkdir: cannot create directory ‘forNotify’: File exists


In [18]:
!g++ notify.c -o notify

notify.c: In function ‘int main()’:
notify.c:17:8: error: ‘inotify_init’ was not declared in this scope
   fd = inotify_init();
        ^~~~~~~~~~~~
notify.c:17:8: note: suggested alternative: ‘inotify_event’
   fd = inotify_init();
        ^~~~~~~~~~~~
        inotify_event
notify.c:25:8: error: ‘inotify_add_watch’ was not declared in this scope
   wd = inotify_add_watch( fd, "/forNotify", IN_CREATE | IN_DELETE );
        ^~~~~~~~~~~~~~~~~
notify.c:25:8: note: suggested alternative: ‘inotify_event’
   wd = inotify_add_watch( fd, "/forNotify", IN_CREATE | IN_DELETE );
        ^~~~~~~~~~~~~~~~~
        inotify_event
notify.c:29:12: error: ‘read’ was not declared in this scope
   length = read( fd, buffer, EVENT_BUF_LEN );
            ^~~~
notify.c:29:12: note: suggested alternative: ‘fread’
   length = read( fd, buffer, EVENT_BUF_LEN );
            ^~~~
            fread
notify.c:57:4: error: ‘inotify_rm_watch’ was not declared in this scope
    inotify_rm_watch( fd, wd );
    ^~~~~~~~~

## LAB 7

### 1

In [24]:
%%file sumNum.c
#include <stdio.h>

int main(int argc, char *argv[]){
	int a,b,sum;
	if(argc!=3)
	{
		printf("please use \"prg_name value1 value2 \"\n");
		return -1;
	}
	
	a = atoi(argv[1]);
	b = atoi(argv[2]);
	sum = a+b;
	
	printf("Sum of %d, %d is: %d\n",a,b,sum);
	
	return 0;
}

Writing sumNum.c


In [27]:
!g++ sumNum.c

sumNum.c: In function ‘int main(int, char**)’:
sumNum.c:11:6: error: ‘atoi’ was not declared in this scope
  a = atoi(argv[1]);
      ^~~~
sumNum.c:11:6: note: suggested alternative: ‘auto’
  a = atoi(argv[1]);
      ^~~~
      auto


## 2

## LAB 8

In [21]:
%%file calendar_time.c
#include <locale.h>
#include <time.h>
#include <sys/time.h>
//#include "tlpi_hdr.h"

#define SECONDS_IN_TROPICAL_YEAR (365.24219 * 24 * 60 * 60)

int main(int argc, char *argv[]){
    time_t t;
    struct tm *gmp, *locp;
    struct tm gm, loc;
    struct timeval tv;

    /* Retrieve time, convert and display it in various forms */

    t = time(NULL);
    printf("Seconds since the Epoch (1 Jan 1970): %ld", (long) t);
    printf(" (about %6.3f years)\n", t / SECONDS_IN_TROPICAL_YEAR);

    if (gettimeofday(&tv, NULL) == -1)
        errExit("gettimeofday");
    printf("  gettimeofday() returned %ld secs, %ld microsecs\n",
            (long) tv.tv_sec, (long) tv.tv_usec);

    gmp = gmtime(&t);
    if (gmp == NULL)
        errExit("gmtime");

    gm = *gmp;          /* Save local copy, since *gmp may be modified
                           by asctime() or gmtime() */
    printf("Broken down by gmtime():\n");
    printf("  year=%d mon=%d mday=%d hour=%d min=%d sec=%d ", gm.tm_year,
            gm.tm_mon, gm.tm_mday, gm.tm_hour, gm.tm_min, gm.tm_sec);
    printf("wday=%d yday=%d isdst=%d\n", gm.tm_wday, gm.tm_yday, gm.tm_isdst);

    /* The TZ environment variable will affect localtime().
       Try, for example:
                TZ=Pacific/Auckland calendar_time
    */

    locp = localtime(&t);
    if (locp == NULL)
        errExit("localtime");

    loc = *locp;        /* Save local copy */

    printf("Broken down by localtime():\n");
    printf("  year=%d mon=%d mday=%d hour=%d min=%d sec=%d ",
            loc.tm_year, loc.tm_mon, loc.tm_mday,
            loc.tm_hour, loc.tm_min, loc.tm_sec);
    printf("wday=%d yday=%d isdst=%d\n\n",
            loc.tm_wday, loc.tm_yday, loc.tm_isdst);

    printf("asctime() formats the gmtime() value as: %s", asctime(&gm));
    printf("ctime() formats the time() value as:     %s", ctime(&t));

    printf("mktime() of gmtime() value:    %ld secs\n", (long) mktime(&gm));
    printf("mktime() of localtime() value: %ld secs\n", (long) mktime(&loc));

    exit(EXIT_SUCCESS);
}

Overwriting calendar_time.c


In [22]:
!gcc calendar_time.c -o calendar_time

calendar_time.c: In function ‘main’:
calendar_time.c:17:5: warning: implicit declaration of function ‘printf’ [-Wimplicit-function-declaration]
     printf("Seconds since the Epoch (1 Jan 1970): %ld", (long) t);
     ^~~~~~
calendar_time.c:17:5: warning: incompatible implicit declaration of built-in function ‘printf’
calendar_time.c:17:5: note: include ‘<stdio.h>’ or provide a declaration of ‘printf’
calendar_time.c:21:9: warning: implicit declaration of function ‘errExit’; did you mean ‘_Exit’? [-Wimplicit-function-declaration]
         errExit("gettimeofday");
         ^~~~~~~
         _Exit
calendar_time.c:60:5: warning: implicit declaration of function ‘exit’ [-Wimplicit-function-declaration]
     exit(EXIT_SUCCESS);
     ^~~~
calendar_time.c:60:5: warning: incompatible implicit declaration of built-in function ‘exit’
calendar_time.c:60:5: note: include ‘<stdlib.h>’ or provide a declaration of ‘exit’
calendar_time.c:60:10: error: ‘EXIT_SUCCESS’ undeclared (first use in this functio